# Domaca naloga 1.2

Avtomatizirana izbira najbosega modela in nastavitev hiper parametrov.


In [1]:
import pandas as pd
import numpy as np

from hyperopt import hp, tpe, rand, fmin, Trials, space_eval
from hyperopt import pyll, base

import warnings

# ne izpisuje opozoril
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

data = pd.read_csv('podatki.csv')
X = data.drop('y', axis=1)
Y = data['y']
clas = Y.unique()
Y = Y.replace(clas[0], 0)
Y = Y.replace(clas[1], 1)

#print(Y)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_train_sc = pd.DataFrame(X_train_sc,columns=X.columns)
X_test_sc = scaler.transform(X_test)
X_test_sc = pd.DataFrame(X_test_sc,columns=X.columns)



Prvi poskus se je izkazal da zadeva predolgo melje zato sem se odlocil za mansanje dimenzij podatkov. Izbral sem pristop **feature_importances_** tako da sem izbral prvih 7 najpomebnejsih znacilk, ker je pri tej znacilki AUC presegel vrednost 0.8.

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate


np.random.seed(0)

rfo = RandomForestClassifier().fit(X_train, y_train)

col_name = X.columns
imprf = rfo.feature_importances_


sortind = np.argsort(-imprf)
sort_imprf = imprf[sortind]
sort_name = col_name[sortind]
#print(X_train[sort_name[0:0]])

for i in range(1, len(sort_name)):
    x_train = X_train[sort_name[0:i]]
    x_test = X_test[sort_name[0:i]]
    rf_imp = RandomForestClassifier()
    cv = cross_validate(rf_imp, x_train, y_train, cv=5,
                        scoring=('roc_auc'),
                        return_train_score=True)
    rf_imp.fit(x_train, y_train)
    pred = rf_imp.predict(x_test)
    print(f"cv = {np.mean(cv['test_score'])}")
    print(f"test = {roc_auc_score(pred, y_test)}")
    if roc_auc_score(pred, y_test) > 0.8:
        print(i)
        break
    
print(sort_name[0:i])


cv = 0.863463708557602
test = 0.7438640654499685
cv = 0.8569429683217973
test = 0.6790650406504065
cv = 0.890874744172675
test = 0.702527646129542
cv = 0.9126591789581873
test = 0.7174418604651163
cv = 0.9186002122358963
test = 0.7957535885167464
cv = 0.9173449704734751
test = 0.7499074074074075
cv = 0.9276619113121081
test = 0.8087182823682498
7
Index(['x1', 'x5', 'x3', 'x2', 'x4', 'x21', 'x27'], dtype='object')


In [3]:
gnezdeni_prostor = {
    "algo": hp.choice('algo', [
        {
            'name': 'rf',
            'max_depth': hp.choice('max_depth', list(range(2, 50, 5))),
            'min_samples_split' : hp.choice('min_samples_split', list(range(1, 20, 2))),
            'min_samples_leaf' : hp.choice('min_samples_leaf', [1, 2, 3])

        },
        {
            'name': 'knn',
            'n_neighbors' : hp.choice('n_neighbors', list(range(2, 30, 2)))
        },
        {
            'name': 'svm',
            'C': hp.choice('C', [1.0, 10.0]),
            'kernel': hp.choice('kernel', [
                {
                    'type': 'linear'  # linearno
                },
                {
                    'type': 'sigmoid'
                },
                {
                    'type': 'poly'
                }
            ]),
        },
    ])
}


In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate

np.random.seed(0)

i = 7
#Xi_train = X_train[sort_name[0:i]]
#Xi_test =  X_test[sort_name[0:i]]

Xi_train_sc = X_train_sc[sort_name[0:i]]
Xi_test_sc =  X_test_sc[sort_name[0:i]]

def kriteri_fn(params):
    a = params["algo"]
    algo_name = a["name"]
    if algo_name == "knn":
        model = KNeighborsClassifier(n_neighbors=a["n_neighbors"])

    elif algo_name == "rf":
        model = RandomForestClassifier(max_depth=a['max_depth'], 
                                       min_samples_split=a['min_samples_split'], 
                                       min_samples_leaf=a['min_samples_leaf'])

    elif algo_name == "svm":
        kernel = a["kernel"]["type"]
        C = a['C']
        model = SVC(kernel=kernel, C=C)
    else:
        raise ValueError("Wrong setings!")
    
    cv = cross_validate(model, Xi_train_sc, y_train, cv=5,
                        scoring=('roc_auc'),
                        return_train_score=True)

    return 1 - np.mean(cv['test_score'])


trials = Trials()
best = fmin(
            fn = kriteri_fn,
            space = gnezdeni_prostor,
            algo = tpe.suggest,
            max_evals = 500,
            trials = trials
            )
print(best)
best_m = space_eval(gnezdeni_prostor, best)
best_value = kriteri_fn(best_m)




100%|██████████| 500/500 [02:14<00:00,  3.73trial/s, best loss: 0.06291755946636424]
{'algo': 0, 'max_depth': 1, 'min_samples_leaf': 0, 'min_samples_split': 6}


In [5]:
print(f"Best algo with params {best}")

Best algo with params {'algo': 0, 'max_depth': 1, 'min_samples_leaf': 0, 'min_samples_split': 6}


In [10]:
print(trials.trials)
s = [print(trials.trials[i]) for i in range(len(trials.results))]



Sedaj pa se nataniram model z vsemi znacilkami in izracunanimi parametri

In [14]:

Xi_train = X_train[sort_name[0:10]]
Xi_test =  X_test[sort_name[0:10]]

np.random.seed(0)

bestmodel = RandomForestClassifier(max_depth=7, min_samples_leaf=1, min_samples_split=6)
cv = cross_validate(bestmodel, Xi_train, y_train, cv=5,
                        scoring=('roc_auc'),
                        return_train_score=True)

bestmodel.fit(Xi_train, y_train)
pred = bestmodel.predict(Xi_test)

print(f"AUC on cv = {np.mean(cv['test_score'])}")
print(f"AUC on test = {roc_auc_score(pred, y_test)}")



AUC on cv = 0.9321965804448158
AUC on test = 0.8385222083852221
